## init 

In [ ]:
!pip uninstall timer-decorator

In [ ]:
import execution_timer

In [24]:
%load_ext autoreload
%autoreload 2

import chess
import ipywidgets as widgets
import chess.pgn
import numpy as np
import io
import sys



if sys.platform=='win32':
    STOCKFISHPATH=r'c:/gitproj/stockfish/stockfish-windows-x86-64-avx2.exe'
else:
    STOCKFISHPATH='/home/jovyan/stockfish/src/stockfish'

engine = chess.engine.SimpleEngine.popen_uci(STOCKFISHPATH)

#this after engine
import nest_asyncio
nest_asyncio.apply()

from ipywidgets import interact, interactive, fixed, interact_manual, SelectMultiple, Combobox, HBox, VBox
import ipywidgets as widgets
import chess.engine
mycalc=None
global mycalc

from calc import Calculator
from chess._interactive import InteractiveViewer
from IPython.display import clear_output
import asyncio
from random import randrange
gbuf=""
class GameDispCalc(InteractiveViewer):    
    @staticmethod
    def get_game(pgn):
        p=chess.pgn.read_game(io.StringIO(pgn))
        gam=p.game()

        return gam 
    @classmethod
    def get_me(cls,pgn):
        iv=GameDispCalc.get_game(pgn)._interactive_viewer()
        iv.__class__=cls
        iv.patch()
        return iv

    def patch(self):
        from copy import deepcopy
        gam= deepcopy(self.game)

        ls=[]
        for i,k in enumerate(self._InteractiveViewer__moves):
            san=gam.board().san(k)
            gam=gam.next()
            ls+=[san]
        self._InteractiveViewer__white_moves = [str(move) for (i, move) in enumerate(ls) if i % 2 == 0]
        self._InteractiveViewer__black_moves = [str(move) for (i, move) in enumerate(ls) if i % 2 == 1]
        self.moves=ls
        self.calc= Calculator( STOCKFISHPATH)
        self.task=None
        
        self.w= widgets.Output(layout={'border': '1px solid black'}) #widgets.interactive_output(f, {'a': 1, 'b': 2, 'c': 3})
    
    async def create_task(self,b):
        #if asyncio.get_event_loop() is None:
        #    asyncio.set_event_loop(asyncio.new_event_loop())

        #def upd(res):
        global gbuf
        gbuf+="ab"
        
        res = await self.calc.async_ret_stats(b,b.turn)

        #self.w= widgets.Output(layout={'border': '1px solid black'})
        nm=self._InteractiveViewer__next_move
        #self.w.clear_output(True)
        if nm>0:
            self.w.append_stdout(self.moves[nm-1])
        #self.w.append_stdout(str(randrange(3,100)))
        
        self.w.append_stdout(str(res))
        #InteractiveViewer.show(self)
        #with self._InteractiveViewer__out:
        #    display('bbb')
        #self._InteractiveViewer__out.            display(self.w)
            
        #if self.w:
        #    self.w.clear_output()
            #self._InteractiveViewer__out.outputs=self._InteractiveViewer__out.outputs[:-1] #+ [ w]
        #self.w=w
        #with self._InteractiveViewer__out:
        #    display(self.w)
        #self.show(fut=False)
        #elf._InteractiveViewer__out.append_display_data(self.w)
        #self._InteractiveViewer__out.clear_output()
        self.show(False)
        #display(self.w)
       
        
        
            
            #print(str(res))
            #print(randrange(3,100))
            #gbuf+=str(randrange(3,100))
            #gbuf+=str(res)
        #print("fff")
        #st=asyncio.run()
        #self.task.add_done_callback(upd) 
        
    def show(self,fut=True):
        global gbuf
        clear_output()
        #self.calc.observe= lambda x: 
        #self._InteractiveViewer__white_moves = [str(move) for (i, move) in enumerate(self._InteractiveViewer__moves) if i % 2 == 0]
        #self._InteractiveViewer__black_moves = [str(move) for (i, move) in enumerate(self._InteractiveViewer__moves) if i % 2 == 1]
        #self._InteractiveViewer__out.append_stdout('bbbb')
        InteractiveViewer.show(self)
        #self._InteractiveViewer__out.append_stdout("fff")
        #from IPython.display import clear_output
        
        
        #if self.task:
        #    self.task.cancel()
        b=self.game.board()
        loop = asyncio.get_event_loop()
        #loop.run_in_executor(None, self.create_task)
        #asyncio.run(self.task)
        
        
        #self._InteractiveViewer__out.outputs=self._InteractiveViewer__out.outputs[:-1]
        
        b=self._InteractiveViewer__board
        # with self._InteractiveViewer__out:
        #     try:
        #         y=self._InteractiveViewer__out.outputs[-1]['data']['text/plain'].startswith("Output")
        #         if y:
        #             self._InteractiveViewer__out.outputs=iv._InteractiveViewer__out.outputs[:-1]
        #             gbuf+=('bfghfgh')
        #     except:
        #         pass
        #     try:
        #         y=self._InteractiveViewer__out.outputs[-1]['data']['text/plain'].startswith("Output")
        #         if y:
        #             self._InteractiveViewer__out.outputs=iv._InteractiveViewer__out.outputs[:-1]
        #             gbuf+=('aaa')
        #     except:
        #         pass
        if fut:
            self.w= widgets.Output(layout={'border': '1px solid black'})
        with self._InteractiveViewer__out:
            display(self.w)
                
        # def tt(xx):
        #     with self._InteractiveViewer__out:
        #         display('bbb')
        #     with self._InteractiveViewer__out:
        #         display(self.w)
        if fut:
            self.task=asyncio.create_task(self.create_task(b))
            #self.task.add_done_callback(tt)
                
        
        #print('hhgf')
        #self._InteractiveViewer__out.append_display_data(self.w)
        #return self._InteractiveViewer__out
        
    import asyncio
    def wait_for_change(widget, value):
        future = asyncio.Future()
        def getvalue(change):
            # make the new value available
            future.set_result(change.new)
            widget.unobserve(getvalue, value)
        widget.observe(getvalue, value)
        return future
        
    
    
        
        
        
            
            
        
#     def display_game(self):      
#         pgn=self.pgn
#         def rev(x):
#             a.value=(a.value-1)
#         def adv(x):
#             a.value=(a.value+1)

#         from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
#         gam = GameDispCalc.get_game(pgn)

#         butl=Layout(width='35px',justify_content='center')
#         ba=widgets.Button(description='<',layout=butl)
#         ba.on_click(rev)
#         ne=widgets.Button(description='>',layout=butl)
#         ne.on_click(adv)
#         mmax=len(list(gam.mainline_moves()))
#         a = widgets.IntSlider(value=0, min=0, max=mmax)
#         items = [ba,ne, widgets.Button(description='R',layout=butl),a]


#         gb2 = widgets.HBox(items,layout=Layout(display='flex',
#                 flex_flow='row',
#                 align_items='stretch',
#                 width='100%'))
#         def f2(a):
#             #update game progress
#             cur=gam
#             for k in range(a):
#                 cur=cur.next()
#             display( widgets.HTML(str(cur.board()._repr_svg_())))


#         def f3(a):
#             #update stability calc
#             cur=gam
#             for k in range(a):
#                 cur=cur.next()

#             iswhite= (a%2 ==0)

#             curb=cur.board()
#             #display(list(curb.generate_legal_moves()))

#             self.print_stats(curb,iswhite,full=(a>5))

#         return VBox([gb2,widgets.interactive_output(f2,{'a':a}),widgets.interactive_output(f3,{'a':a})],layout=Layout(
#             display='flex',
#             flex_flow='column',
#             align_items='stretch',
#             width='90%'
#        ) )
def display_game(pgn):
    return GameDispCalc(pgn).display_game()

def display_board(fen,iswhite=None,simp=False):
    mycalc=Calculator(STOCKFISHPATH)
    b=chess.Board(fen)
    if iswhite is None:
        iswhite= b.turn
    display( widgets.HTML(str(b._repr_svg_())))
    if not simp:
        mycalc.print_stats(b,iswhite)
        mycalc.printtimer() 
    return mycalc
    
fen='r2Rr1k1/pp3pp1/2p2bbp/4n3/4pBP1/2Q1P1NP/PPP2P2/1K3B1R b - - 0 17'
#mycalc=Calculator(engine,None)
b=chess.Board(fen)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
dir(iv._InteractiveViewer__out)

['_Output__counter',
 '__annotations__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_active_widgets',
 '_add_notifiers',
 '_all_trait_default_generators',
 '_append_stream_output',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_control_comm',
 '_cross_validation_lock',
 '_default_keys',
 '_descriptors',
 '_dom_classes',
 '_flush',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_control_comm_msg',
 '_handle_custom_msg',
 '_handle_msg',
 '_holding_sync',
 '_instance_inits',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 

In [14]:
iv._InteractiveViewer__out.outputs

({'output_type': 'display_data',
  'data': {'text/plain': "GridBox(children=(Button(icon='step-forward', layout=Layout(grid_area='right', width='60px'), style=ButtonStyl…",
   'application/vnd.jupyter.widget-view+json': {'version_major': 2,
    'version_minor': 0,
    'model_id': 'b4f15e48a3b84e9a8f4ba1e3f80957f7'}},
  'metadata': {}},)

In [6]:
import ipywidgets

In [7]:
ipywidgets

<module 'ipywidgets' from 'C:\\Users\\ekarni\\.pyenv\\pyenv-win\\versions\\3.10\\lib\\site-packages\\ipywidgets\\__init__.py'>

In [16]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future
from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider


IntSlider(value=0)

In [17]:
out

Output(outputs=({'output_type': 'stream', 'name': 'stdout', 'text': 'did work 0\n'},))

In [14]:
async def async_function(b):
    await asyncio.sleep(5)
    print("Async function completed")
    b('aa')
    
class P:
    def observe(a,b,c):
         print('zz')
         t=asyncio.create_task(async_function(b))
    @property
    def value(self):
        print("xx")
        
def f(a):
    print(a)
widgets.interactive_output(f, {'a': P()} )

zz
xx


Output()

In [18]:
import asyncio
# Define an async function
async def async_function():
    await asyncio.sleep(5)
    print("Async function completed")
# Define a regular function
def regular_function():
    loop = asyncio.get_event_loop()
    t=asyncio.create_task(async_function())
    #loop.run_in_executor(None, async_function())
    #loop.call_soon(asyncio.ensure_future(t))
    #loop.run_forever()
    
    print('bbb')
# Call the regular function
#regular_function()


In [3]:
dd=iv._InteractiveViewer__out

In [4]:
dd.notify_ch

Signature: dd.notify_change(change)
Docstring: Called when a property has changed.
File:      c:\users\ekarni\.pyenv\pyenv-win\versions\3.10\lib\site-packages\ipywidgets\widgets\widget.py
Type:      method

In [8]:
iv._InteractiveViewer__out

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "GridBox(children=(Button(icon='step-fo…

In [45]:
iv.show()

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "GridBox(children=(Button(icon='step-fo…

In [ ]:
display_board('r2r2k1/pp3pp1/2p2bbp/4n3/4NBP1/2Q1P2P/PPP2P2/1K3B1R b - - 0 18)

In [ ]:
display_board(fen)

In [ ]:
ff='3rr1k1/pp3pp1/2p2bbp/4n3/4pBP1/2Q1P1NP/PPP2P2/1K3B1R w - - 0 18'

In [ ]:
chess.Board(ff)

In [ ]:
b

In [ ]:
deprel=1/3
f= 2** (-10* deprel)
l=chess.engine.Limit(depth=3, time=0.001 * f * 1000  ,nodes=800 * f * 1000)
engine.analyse(b,l)

In [ ]:
deprel=1/3
f= 2** (-10* deprel)
l=chess.engine.Limit(depth=3, time=0.001 * f * 1000  ,nodes=800 * f * 1000)
engine.analyse(chess.Board(ff),l)

In [ ]:
uci.from_fen(*fen.split(" "))

In [ ]:
fen =='r3R1k1/pp3pp1/2p2bbp/8/4pBP1/2Q1P1NP/PPP2P2/1K2nB1R w - - 1 17'

In [ ]:
b

In [ ]:
display_board ('r3R1k1/pp3pp1/2p2bbp/8/4pBP1/2Q1P1NP/PnP2P2/1K3B1R w - - 0 19',simp=True)

In [ ]:
display_board('r2R2k1/pp2rpp1/2p2bbp/4n3/4pBP1/2Q1P1NP/PPP2P2/1K3B1R w - - 1 18',simp=True)

In [ ]:
display_board(fen)

In [ ]:
!pip uninstall chess

In [ ]:
!pip install chess

In [ ]:
gam

In [ ]:

fen.split(' ')[1]

## Carlsen game

In [25]:
pgn='''[Event "CCT Finals KO 2023"]
[White "Carlsen,Magnus"]
[Black "Caruana,Fabiano"]
[Site "Toronto"]
[Round "1.1"]
[Annotator "TA"]
[Result "1-0"]
[Date "2023.12.13"]
[WhiteElo "2830"]
[BlackElo "2794"]
[PlyCount "181"]
[TimeControl "900+3"]

{[%evp 25,181,0,0,0,0,-1,0,0,7,13,7,0,20,-8,6,0,0,0,0,0,0,0,77,0,76,59,61,72,193,0,0,0,8,8,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-41,0,0,0,0,0,0,0,0,0,0,0,0,0,-243,-225,-179,-120,-136,-130,-564,-489,-719,-767,-838,-613,-580,-548,-448,-188,-256,-262,-412,-234,-279,-266,-491,-527,-493,-454,-401,-389,-391,0,-831,0,0,0,0,0,0,0,-29989,-29990,-29990,0,-1612,-1449,-29990,-29976,-6070,-5081,-4308,-4704,-5069,0,0,0,-5072,-5057,-1446,0,0,0,0,0,0,0,0,0,0,0,0,29991,29992]} 1. e4 {[%emt 0:0:3]} e5 {[%emt 0:0:1]} 2. Nf3 {[%emt 0:0:2]} Nc6 {[%emt 0:0:2]} 3. Bb5 {[%emt 0:0:2]} a6 {[%emt 0:0:2]} 4. Ba4 {[%emt 0:0:1]} Nf6 {[%emt 0:0:2]} 5. O-O {[%emt 0:0:4]} Bc5 {[%emt 0:0:2]} 6. c3 {[%emt 0:0:41]} b5 {[%emt 0:0:4]} 7. Bb3 {[%emt 0:0:2]} d6 {[%emt 0:0:2]} 8. d4 {[%emt 0:1:7]} Bb6 {[%emt 0:0:2]} 9. Be3 {[%emt 0:0:3]} O-O {[%emt 0:0:8]} 10. Nbd2 {[%emt 0:0:3]} h6 {[%emt 0:0:7]} 11. h3 {[%emt 0:1:7]} Re8 {[%emt 0:0:3]} 12. a3 {[%emt 0:1:32]} Rb8 {[%emt 0:2:48]} 13. Bc2 {[%emt 0:1:11]} a5 {[%emt 0:0:40]} 14. Re1 {[%emt 0:0:15]} a4 {[%emt 0:0:23]} 15. Bd3 {[%emt 0:0:11]} Bd7 {[%emt 0:0:48]} 16. Qe2 {[%emt 0:1:18]} exd4 {[%emt 0:3:5]} 17. cxd4 {[%emt 0:0:3]} Ne7 $1 {[%emt 0:0:1]} 18. Rac1 {[%emt 0:2:3]} Ng6 {[%emt 0:0:24]} 19. Qf1 {[%emt 0:0:17]} Ba5 {[%emt 0:0:36]} 20. Red1 {[%emt 0:0:6]} c6 {[%emt 0:0:25]} 21. e5 {[%emt 0:0:47]} Nd5 {[%emt 0:1:23]} 22. Bxg6 {[%emt 0:0:38]} fxg6 {[%emt 0:0:1]} 23. Ne4 {[%emt 0:0:1]} Rf8 {[%emt 0:1:1]} (23... dxe5 $1 $11 {keeps the balance.} 24. Nxe5 Bc7 )24. Nxd6 {[%emt 0:1:5]} (24. Qe2 $1 $16 )Bxh3 {[%emt 0:0:32]} (24... Rxf3 $1 $11 {remains equal.} 25. gxf3 Qh4 )25. gxh3 {[%emt 0:0:22]} (25. Qd3 {feels hotter.} Ne7 26. Nh4 g5 27. Ng6 )Rxf3 {[%emt 0:0:2]} 26. Qg2 {[%emt 0:0:1]} (26. Rxc6 $2 Nxe3 27. fxe3 Rxf1+ $19 )Nxe3 {[%emt 0:0:22]} (26... Rxe3 $16 27. fxe3 Nxe3 28. Qxg6 Nxd1 29. Rxd1 Qg5+ 30. Qxg5 hxg5 )27. Qxf3 $2 {[%emt 0:0:46]} (27. fxe3 $18 Qg5 28. Qxg5 (28. Rxc6 $2 Qxe3+ 29. Kh2 Rf2 $19 )hxg5 29. e4 (29. Rxc6 Rxe3 30. Rdc1 Rf8 $14 ))Nxd1 $11 {[%emt 0:0:2]} 28. Rxd1 {[%emt 0:0:2]} Bc7 {[%emt 0:1:30]} 29. Qxc6 {[%emt 0:0:43]} Bxd6 {[%emt 0:0:2]} 30. Qd5+ {[%emt 0:0:31]} Kh7 {[%emt 0:0:2]} 31. exd6 {[%emt 0:0:2]} Qd7 {[%emt 0:0:10]} 32. Kg2 {[%emt 0:0:49]} Rb6 {[%emt 0:0:5]} 33. Qe4 {[%emt 0:0:16]} Rxd6 {[%emt 0:0:2]} 34. d5 {[%emt 0:0:2]} Rf6 {[%emt 0:0:5]} 35. Rd3 {[%emt 0:0:9]} Rd6 {[%emt 0:0:52]} 36. Qe5 {[%emt 0:0:7]} Qd8 {[%emt 0:0:7]} 37. Kf1 {[%emt 0:0:13]} Qc7 {[%emt 0:0:15]} 38. Rc3 {[%emt 0:0:7]} Qd7 {[%emt 0:0:5]} 39. Rd3 {[%emt 0:0:0]} Qd8 {[%emt 0:0:10]} 40. Ke2 {[%emt 0:0:3]} Qb6 {[%emt 0:0:6]} 41. Qd4 {[%emt 0:0:5]} Qb7 {[%emt 0:0:2]} 42. Kd2 {[%emt 0:0:1]} Qf7 {[%emt 0:0:4]} 43. Kc2 {[%emt 0:0:2]} Rf6 {[%emt 0:0:5]} 44. f3 {[%emt 0:0:4]} Qc7+ {[%emt 0:0:2]} 45. Kb1 {[%emt 0:0:2]} Qd6 {[%emt 0:0:5]} 46. Ka2 {[%emt 0:0:5]} Rf4 {[%emt 0:0:1]} 47. Qe3 {[%emt 0:0:4]} b4 {[%emt 0:0:3]} 48. axb4 $1 {[%emt 0:0:1]} Rxb4 {[%emt 0:0:1]} 49. Qd2 {[%emt 0:0:4]} Rb8 {[%emt 0:0:7]} 50. Rd4 {[%emt 0:0:1]} Rb3 {[%emt 0:0:12]} 51. Rd3 {[%emt 0:0:7]} Rb8 {[%emt 0:0:3]} 52. Rd4 {[%emt 0:0:1]} Rb5 {[%emt 0:0:3]} 53. h4 {[%emt 0:0:4]} h5 {[%emt 0:0:1]} 54. Qd3 {[%emt 0:0:5]} Rb3 {[%emt 0:0:2]} 55. Qd1 {[%emt 0:0:0]} Qb6 {[%emt 0:0:5]} 56. Qd2 $2 {[%emt 0:0:2]} (56. Rd2 $11 {and White stays safe.} )Rxf3 $19 {[%emt 0:0:3]} (56... a3 57. bxa3 Rxf3 58. Qb4 Rf2+ 59. Kb3 )57. Rd3 {[%emt 0:0:0]} Rf2 {[%emt 0:0:4]} (57... Rf1 $1 $19 58. Qe3 Qxe3 59. Rxe3 Rd1 )58. Qc3 $17 {[%emt 0:0:1]} Qd6 $1 {[%emt 0:0:4]} 59. Qc6 $2 {[%emt 0:0:2][%mdl 8192]} (59. Rd4 $17 {is more resistant.} )Qh2 $19 {[%emt 0:0:5]} 60. Qb6 {[%emt 0:0:2]} (60. Qc1 $142 a3 61. Rxa3 )Qxh4 $19 {[%emt 0:0:4]} 61. Rd4 {[%emt 0:0:2]} Qg3 {[%emt 0:0:4]} 62. Qb4 {[%emt 0:0:4]} a3 {[%emt 0:0:2]} 63. Qxa3 {[%emt 0:0:1]} Qe5 {[%emt 0:0:2]} (63... Qxa3+ $142 64. Kxa3 Rf8 )64. Qd3 {[%emt 0:0:2]} (64. Qb4 {keeps fighting.} )Qd6 $19 {[%emt 0:0:2]} (64... Qb8 $2 65. Qa3 $19 )65. Ra4 {[%emt 0:0:4]} (65. Kb1 {was worth a try.} )Rf5 {[%emt 0:0:13]} (65... Qh2 $142 66. Qc3 Rc2 )66. Rd4 $19 {[%emt 0:0:3]} Rf8 {[%emt 0:0:0]} 67. b4 $2 {[%emt 0:0:1]} (67. Kb1 {is a better defense.} )Rf2+ $19 {[%emt 0:0:0]} 68. Kb3 {[%emt 0:0:1]} Qf6 {[%emt 0:0:0]} 69. Qc4 {[%emt 0:0:8]} Rf3+ {[%emt 0:0:1]} 70. Rd3 {[%emt 0:0:1]} Rf2 $2 {[%emt 0:0:2]} (70... Rf1 $19 {aiming for ...Qa1.} 71. Rc3 Rd1 )71. d6 $2 {[%emt 0:0:2][%mdl 8192]} (71. Rd4 $11 )Qb2+ $2 {[%emt 0:0:6]} ({Black should try} 71... Qa1 $19 {( -> ...Qa2+)} 72. Qc5 Qa2+ 73. Kc3 Rc2+ 74. Kd4 Qa1+ 75. Ke4 Qh1+ 76. Ke5 Qh2+ 77. Ke4 Qg2+ 78. Rf3 Re2+ 79. Qe3 h4 )72. Ka4 $11 {[%emt 0:0:1]} Qa1+ {[%emt 0:0:1]} 73. Kb5 {[%emt 0:0:1]} Qa8 {[%emt 0:0:7]} (73... Rf8 $5 74. d7 $8 Rf6 $11 )74. d7 $1 {[%emt 0:0:2]} Rf5+ {[%emt 0:0:0]} 75. Kb6 $2 {[%emt 0:0:1][%mdl 8192]} (75. Qc5 $11 Qb7+ 76. Ka5 Qa8+ 77. Kb5 Rxc5+ 78. bxc5 )Qb8+ {[%emt 0:0:2]} 76. Ka6 {[%emt 0:0:1]} Qa8+ {[%emt 0:0:2]} (76... Rf6+ $142 77. Ka5 Qa7+ 78. Kb5 Rb6+ 79. Kc5 Re6+ 80. Kd5 Qxd7+ 81. Kc5 Rc6+ 82. Kb5 Rd6+ {[%mdl 64] Discovered Attack} 83. Ka5 Qa7+ 84. Kb5 Qa6+ 85. Kc5 Qc6# )77. Kb6 $19 {[%emt 0:0:1] [#]} Rf6+ $1 {[%emt 0:0:5]} 78. Kc5 {[%emt 0:0:23]} Rc6+ {[%emt 0:0:2]} 79. Kd4 {[%emt 0:0:1]} Rxc4+ {[%emt 0:0:4]} 80. Kxc4 {[%emt 0:0:1]} Qd8 {[%emt 0:0:8]} 81. b5 {[%emt 0:0:1]} Qc7+ $2 {[%emt 0:0:3]} (81... g5 $19 82. b6 g4 )82. Kb3 $11 {[%emt 0:0:2]} Qd8 {[%emt 0:0:1]} 83. Ka4 $2 {[%emt 0:0:7][%mdl 8192]} (83. Kc4 $11 g5 84. b6 )h4 $19 {[%emt 0:0:4]} 84. b6 {[%emt 0:0:0] [#]} h3 $2 {[%emt 0:0:4]} ({And not} 84... Qxb6 85. d8=Q (85. d8=B {gets mated.} Qa6+ 86. Kb4 Qxd3 87. Bxh4 Qd4+ 88. Kb5 Qxh4 89. Kc6 Qe4+ 90. Kb6 g5 91. Kc7 g4 92. Kd6 g3 93. Kc5 g2 94. Kd6 g1=Q 95. Kd7 Qa7+ 96. Kd8 Qea8# )Qxd8 86. Rxd8 $11 )(84... g5 $1 $19 {[%cal Rg5g4] ...g4! would be deadly.} 85. b7 g4 )85. Rxh3+ $11 {[%emt 0:0:1]} Kg8 {[%emt 0:0:0]} 86. Rd3 {[%emt 0:0:2]} Kf7 {[%emt 0:0:1]} (86... Qxb6 87. d8=Q+ (87. d8=R+ $2 {loses.} Kf7 $19 )Qxd8 88. Rxd8+ $18 )87. b7 {[%emt 0:0:1]} Ke7 {[%emt 0:0:0]} 88. Kb5 {[%emt 0:0:1]} g5 {[%emt 0:0:3]} 89. Kc6 {[%emt 0:0:4]} g4 {[%emt 0:0:2]} 90. Re3+ {[%emt 0:0:13]} Kf6 $2 {[%emt 0:0:4][%mdl 8192]} (90... Kf7 $11 )91. Re8 {[%emt 0:0:2]} 1-0

'''
#display_game(pgn)

iv=GameDispCalc.get_me(pgn)

#iv.__class__ = GameDispCalc

iv.show()



Output()

In [10]:
tt=iv._InteractiveViewer__out

In [14]:
with tt:
    print(tt.)

In [16]:
tt.outputs[0]

{'output_type': 'display_data',
 'data': {'text/plain': "GridBox(children=(Button(icon='step-forward', layout=Layout(grid_area='right', width='60px'), style=ButtonStyl…",
  'application/vnd.jupyter.widget-view+json': {'version_major': 2,
   'version_minor': 0,
   'model_id': ''}},
 'metadata': {}}

In [7]:
gbuf

'ababbfghfghabbfghfghabbfghfghabbfghfghabbfghfghabbfghfghabbfghfghabbfghfghab'

In [33]:
iv.

'ab4score: -0.42\nstability factor: 98.68%\nnum of reasonable moves: 1366\nmax(score) of reasonable: 2.13\nmin(score) of reasonable: -0.6\nfaraction method: False\nmoves by depth: {1: 18, 2: 296, 3: 1366}\nab37score: -0.42\nstability factor: 98.68%\nnum of reasonable moves: 1366\nmax(score) of reasonable: 2.13\nmin(score) of reasonable: -0.6\nfaraction method: False\nmoves by depth: {1: 18, 2: 296, 3: 1366}\nab78score: -0.42\nstability factor: 98.68%\nnum of reasonable moves: 1366\nmax(score) of reasonable: 2.13\nmin(score) of reasonable: -0.6\nfaraction method: False\nmoves by depth: {1: 18, 2: 296, 3: 1366}\nab84score: -0.42\nstability factor: 98.68%\nnum of reasonable moves: 1366\nmax(score) of reasonable: 2.13\nmin(score) of reasonable: -0.6\nfaraction method: False\nmoves by depth: {1: 18, 2: 296, 3: 1366}\n'

In [63]:
iv._Int

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "GridBox(children=(Button(icon='step-fo…

In [11]:
gg=GameDispCalc.get_game(pgn)

<ChildNode at 0x1b8342d0e80 (1. e4 ...)>

In [2]:
pgn='''[Event "Biel Chess Festival"]
[Site "Biel SUI"]
[Date "2007.08.02"]
[Round "9"]
[White "Magnus Carlsen"]
[Black "Teimour Radjabov"]
[Result "1-0"]
[EventDate "?"]
[ECO "B07"]
[WhiteElo "2710"]
[BlackElo "2747"]
[PlyCount "55"]

1. e4 d6 2. d4 Nf6 3. Nc3 e5 4. Nge2 Nbd7 5. g3 c6 6. Bg2 b5 7. a3 Be7 8. O-O
O-O 9. h3 a5 10. g4 Ba6 11. Ng3 b4 12. Nce2 bxa3 13. Rxa3 d5 14. Re3 dxe4 15.
Re1 Qc7 16. Nf5 Bd8 17. g5 Nd5 18. Rxe4 f6 19. Neg3 g6 20. Nh6+ Kg7 21. dxe5
fxg5 22. e6 Kxh6 23. e7 Qb6 24. exf8=Q+ Nxf8 25. c4 Nf4 26. Qd6 Kg7 27. Bxf4
gxf4 28. Re7+ 1-0'''
display_game(pgn)

AttributeError: 'GameDispCalc' object has no attribute 'display_game'

In [ ]:
display_board(fen)

## Najdorfg

In [ ]:
naj='''
1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 a6 6. Be2 e6 7. O-O Nbd7 8. f4 b5 9. Bf3 Bb7 10. e5 Bxf3 11. Nxf3 Ng4 12. Qe2 b4 13. Ne4 d5 14. Nd6+ Bxd6 15. exd6 Qb6+ 16. Kh1 '''
display_game(naj)

## my stupid game

In [ ]:
pgn2='''
[Event "Live Chess"]
[Site "Chess.com"]
[Date "2023.12.05"]
[Round "?"]
[White "nevertheless33"]
[Black "andrewmoussa"]
[Result "1-0"]
[ECO "D01"]
[TimeControl "300"]
[EndTime "14:11:45 PST"]
[Termination "nevertheless33 won by resignation"]

1. d4 d5 2. Nc3 Nf6 3. Bg5 Ne4 4. Nxe4 dxe4 5. Qd2 h6 6. Be3 Bf5 7. h3 e6 8. g4
Bg6 9. O-O-O Be7 10. Kb1 O-O 11. Bf4 Nd7 12. Qc3 c6 13. e3 Bf6 14. Ne2 Re8 15.
Ng3 e5 16. dxe5 Nxe5 17. Rxd8 Raxd8 18. Be2 Nc4 19. Qxc4 b5 20. Qxc6 Re6 21. Qc7
Ra6 22. Rd1 Rxd1+ 23. Bxd1 Rb6 24. Bd6 Ra6 25. Qc8+ 1-0'''

display_game(pgn2)

In [ ]:
%run C:\gitproj\chess_analyzer\calc.py

In [ ]:
sys.executable

### Debug

In [ ]:
g=[-0.1, 0.63, -0.35, 0.28, -0.07, 0.73, 0.41, 0.64, 0.21, 0.46, 0.75, 0.2, 0.39, 0.72, 0.6, 0.49, -0.1, 0.31, 0.5, 0.01, 0.39, 0.18, -0.49, 0.9, 0.7, -0.01, 0.39, -0.64, 0.02, -0.88, -0.6, -0.56, -0.26, -0.07, -0.47, -0.3, -0.26, -0.68, -0.19, -0.43, -0.21, -0.19, -0.66, 0.0, -0.04, -0.15, -0.21, -0.51, -1.2, -0.75, -0.92, 0.02, -0.46, -0.33, -0.57, 0.41, -0.21, -0.53, -0.17, 0.05, -0.29, -1.18, 0.16, -0.49, 0.48, -0.13, -0.03, -0.67, 0.26, 0.09, -0.15, 0.09, -0.14, -1.21, -0.27, 0.57, 0.22, -0.69, -0.29, -0.04, 0.45, -0.39, 0.0, -0.1, 0.17, 0.48, -0.11, 0.15, 0.72, -0.25, 0.31, 0.2, 0.26, 0.32, 0.25, 0.29, 0.29, 0.18, 0.36, 0.16, -0.31, -0.25, 0.86, 0.47, -0.64, 0.02, -0.31, 0.16, -0.7, 0.09, -0.42, 0.15, -0.29, -0.03, 0.39, -0.46, 0.25, -0.09, -0.07, 0.19, -0.15, 0.06, -0.02, -0.04, 0.07, -0.22, -0.4, -0.41, 0.57, 0.2, -0.22, -0.23, -0.14, 0.07, -0.93, -0.39, -0.42, -0.17, 0.01, -0.32, -0.41, 0.04, -0.29, 0.05, -0.21, -0.07, -0.11, -0.53, -0.13, -0.21, -0.46, -0.18, -0.15, -0.77, -0.42, 0.52, 0.08, -0.21, -0.12, 0.55, 0.71, -0.48, 0.51, 0.12, 0.26, 0.17, 0.6, 1.05, 0.03, 0.62, 0.58, 0.52, 0.72, 0.26, 0.52, 0.42, 0.35, 0.64, 0.3, -0.21, 0.24, 1.25, 0.58, 0.2, 0.12, -0.47, -0.02, -0.88, -0.49, -0.51, -0.17, -0.45, -0.34, 0.4, -0.78, -0.05, -1.21, -0.17, -0.13, -0.91, -0.01, -0.16, -0.1, -0.13, -0.47, -0.61, -0.52, 0.44, 0.0, -1.07, -0.38, -0.24, 0.14, -0.76, -0.04, -0.27, -0.26, 0.09, -0.21, -0.36, 0.28, -0.45, -0.18, -0.21, 0.09, 0.09, -0.1, 0.18, -0.04, -0.16, 0.05, -0.04, -1.53, -0.42, 0.3, 0.11, -0.39, 0.07, -0.54, 0.22, -0.64, -0.09, -0.46, -0.29, -0.03, -0.38, -0.1, 0.12, -0.43, 0.15, -1.11, -0.01, -0.03, -0.32, 0.16, -0.13, -0.24, -0.12, -0.25, -0.69, -0.47, 0.25, 0.22, -0.53, -0.36, -0.17, 0.13, -0.66, -0.04, -0.37, 0.18, 0.06, -0.21, -0.19, 0.49, -0.58, 0.29, 0.03, -0.05, 0.06, -0.07, 0.1, -0.05, -0.19, 0.01, -0.17, -0.47, -0.47, 0.69, 0.4, -0.49, -0.19, -0.23, 0.13, -0.78, -0.21, -0.38, 0.04, 0.12, -0.13, -0.13, 0.33, -0.54, 0.14, -0.04, -0.16, 0.23, -0.42, 0.02, 0.05, -0.21, -0.08, -0.17, -0.57, -0.56, 0.58, 0.19, -0.52, -0.23, -0.29, 0.57, -0.66, 0.26, -0.06, 0.55, 0.47, -0.03, 0.35, 0.64, -0.1, 0.57, 0.48, 0.48, 0.41, -0.19, 0.54, 0.24, 0.06, 0.39, 0.19, -0.28, 0.89, 0.73, -0.21, 0.18, 0.26, 0.42, 0.7, 0.29, 1.57, 0.5, 0.67, 1.47, 0.89, 0.93, 1.13, 1.16, 1.05, 0.91, 0.82, 1.07, 0.65, 1.34, 1.18, 0.23, 0.7, 1.66, 1.28, 0.51, 1.2, 0.42, 0.4, -0.31, 0.26, 0.0, 0.26, 0.38, 0.27, 0.19, 0.41, -0.21, 0.12, -0.16, 0.12, 0.46, -0.05, 0.28, 0.24, 0.26, 0.36, 0.28, 0.04, -0.03, -0.44, 0.45, 0.04, 0.22, -0.75, -0.56, -0.78, -0.92, -0.33, -0.92, -0.03, 1.0, -0.33, -0.63, -0.4, 0.04, -0.04, -0.84, -0.51, -0.93, -0.19, -0.77, -0.95, -0.79, 0.05, -1.15, -0.46, -0.56, -0.29, -0.03, -0.4, -0.23, -0.2, -0.72, -0.31, -0.21, -0.18, -0.9, 0.0, -0.15, -0.35, -0.19, -0.48, -1.79, -0.79, -0.86, 0.1, -0.85, -0.35, -1.04, -0.27, -1.65, -0.77, -0.82, -1.11, -0.89, -0.67, -1.0, -0.09, -0.92, -1.45, 0.05, 0.32, -0.6, -0.5, -1.07, 0.0, -0.1, -0.79, -0.74, -0.77, -1.06, -1.03, -0.11, -0.16, -0.99, -0.87, 0.2, 0.48, -0.7, 0.35, -0.37, 0.5, 0.38, -0.05, 0.05, -0.06, -0.5, 0.83, -0.05, 0.1, 0.04, 0.08, 0.32, 0.42, -0.06, 0.3, 0.08, -0.21, -0.22, 0.85, 0.19, -0.53, 0.47, -0.28, 0.28, -0.55, -0.21, -0.43, 0.22, 0.2, -0.1, 0.13, 0.18, -0.15, 0.27, -0.09, 0.06, 0.08, -0.05, 0.25, 0.06, -0.14, 0.24, 0.01, -0.29, -0.31, 0.62, 0.36, -0.07]
init=5.75

In [ ]:
 gb=np.array(g,dtype='float64')/(init/100) * FRACFACTOR

In [ ]:
gb

In [ ]:
gc[gc<0]=0

In [ ]:
gc= (-1)*np.square(gb) *SIGMA

In [ ]:
gc

### def board

In [ ]:
display_board('r2Rr1k1/pp3pp1/2p2bbp/4n3/4pBP1/2Q1P1NP/PPP2P2/1K3B1R b - - 0 17',iswhite=False)

In [ ]:
b

In [ ]:
def convit(b,color,mov):
    from calc import conv,convtrans
    f = conv if color=='w' else convtrans
    return b.san(chess.Move.from_uci(f(mov)))

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci(STOCKFISHPATH)

engine.configure( {"UCI_LimitStrength": True, "UCI_Elo": 1720} )
# Use the 'go' method to compute the engine evaluation
result = engine.analyse(b, chess.engine.Limit(depth=20,time=5000),multipv=30)
# Fetch the three best moves and their scores
for line in result:
    print(f'Move: {line[0]}, Score: {line.score()}')


In [ ]:
engine.analyse(b, chess.engine.Limit(depth=5,time=0.001),multipv=30)

In [ ]:
chess.engine.Limit

In [ ]:
result = engine.analyse(b, chess.engine.Limit(depth=3,time=100),multipv=5)


In [ ]:
b

In [ ]:
result

In [ ]:
domoves(b,result[1]['pv'])

In [ ]:
b

In [ ]:
def domoves(b,movl):
    for m in movl:
        try:
            b.push(m)
            display( widgets.HTML(str(b._repr_svg_())))
        except:
            print('ilegal ' , m )
            break

In [ ]:
result

In [ ]:
b

In [ ]:
res3

In [ ]:
engine2 = chess.engine.SimpleEngine.popen_uci(STOCKFISHPATH,setpgrp=True)
engine2.configure({"UCI_LimitStrength": True, "UCI_Elo":1500 })
res3 = engine2.analyse(b, chess.engine.Limit(depth=13,time=1000),multipv=50,)


In [ ]:
chess.engine.Limit?

In [ ]:
len(res3[12]['pv'])

In [ ]:
import pandas
pandas.DataFrame(res3)

In [ ]:
pandas.DataFrame(res)

In [ ]:
len(res3)

In [ ]:
res3

In [ ]:
engine.configure({"Skill Level": 20})
res = engine.analyse(b, chess.engine.Limit(depth=11,time=1000),multipv=500)


In [ ]:
res

In [ ]:
res2 = engine.analyse(b, chess.engine.Limit(depth=11,time=1000),multipv=500)


In [ ]:
res2

In [ ]:
len(res2)

In [ ]:
len(res2)

In [ ]:
len(result)

In [ ]:
result

## subfish

In [ ]:
#import sys
#sys.path+=[r"c:\gitproj\sunfish"]

In [ ]:
from sunfish.tools import uci
from sunfish.tools.uci import *
from sunfish import sunfish

fen='r2Rr1k1/pp3pp1/2p2bbp/4n3/4pBP1/2Q1P1NP/PPP2P2/1K3B1R b - - 0 17'
uci.sunfish=sunfish



pos=uci.from_fen(*fen.split(" "))




hist = [pos] if uci.get_color(pos) == WHITE else [pos.rotate(), pos]

searcher = sunfish.Searcher()



#for depth, gamma, score, mov  in searcher.search(hist):
#    print(f"info depth {depth} score cp {score} nodes {searcher.nodes} mov {mov}")
#    if depth ==7 :
#        break 

In [ ]:
pos

In [ ]:
searcher.top_moves[pos]

In [ ]:
res=searcher.bound(hist[-1], 0, 5, can_null=False)

In [ ]:
searcher

In [ ]:
b

In [ ]:
def convmovlist(b,y):
    list(map(lambda x: convit(b,'w',x[0]),y))

In [ ]:
convit(bb,'w',searcher.top_moves[pp][5][0])

In [ ]:
pp=pos.move(searcher.top_moves[pos][0][0])

In [ ]:
bb=chess.Board( to_fen(pp,'b'))

In [ ]:
bb

In [ ]:
print(convmovlist(bb,searcher.top_moves[pp]))

In [ ]:
pp=pos.move(searcher.top_moves[pos][0][0])

In [ ]:
pp2=pp.move(searcher.top_moves[pp2][0][0])

In [ ]:
pp.score

In [ ]:
searcher.top_moves[pp2][0][0]

In [ ]:
convi

In [ ]:
searcher.top_moves[pp]

In [ ]:
searcher.top_moves[pos]

In [ ]:
%run C:\gitproj\sunfish\src\tools\uci.py
def to_fen(pos,color):
    if color == 'b':
        pos=pos.rotate()
    
    board = pos.board
    fen = []
    empty = 0
    for c in board:
        if c == "\n":
            if empty > 0:
                fen.append(str(empty))
                empty = 0
            fen.append('/')
        if c.isspace():
            continue
        if c == '.':
            empty += 1
        else:
            if empty > 0:
                fen.append(str(empty))
                empty = 0
            fen.append(c)
        
    fen = ''.join(fen)
    if empty > 0:
        fen += str(empty)
    fen += ' '+color
    
    castling = ''
    if pos.wc[0]: castling += 'Q'
    if pos.wc[1]: castling += 'K'
    if pos.bc[0]: castling += 'q'
    if pos.bc[1]: castling += 'k'
    if castling == '': castling = '-'

    fen += ' '+ castling
    fen += ' ' + (sunfish.render(pos.ep) if pos.ep else '-') + ' 0 1'
    return fen.replace("///","").replace("//","")

to_fen(pos,'b')

In [ ]:
split(fen," ")

In [ ]:
pos.bc

In [ ]:
fen

In [ ]:
pos.fen()

In [ ]:
b

In [ ]:
uci.pv(searcher,pos,include_scores=False)

In [ ]:
" ".join(map(str,pos))

In [ ]:
lll=list(map(lambda x: to_fen(x[0],'w') ,res['sequences']))

In [ ]:
display_board( lll[6],simp=True)

In [ ]:
b

In [ ]:
nfen = ' r2Rr1k1/pp3pp1/2p2bbp/4n3/4pBP1/4P1NP/PPP2P2/1K1Q1B1R b - - 0 17'

In [ ]:
display_board(nfen)

In [ ]:
pv(searcher, pos,include_scores=True)

In [ ]:
zz=[(val,convtrans(move)) for val, move in sorted(((pos.value(m), m) for m in pos.gen_moves()), reverse=True)]
    

In [ ]:
b

In [ ]:
zz

In [ ]:
move = chess.Move.from_uci("g1f3")

In [ ]:
move.

In [ ]:
searcher.search?

In [ ]:
conv(mov)

In [ ]:
mov

In [ ]:
display_board(fen)

In [ ]:
def display_board2(fen,iswhite=None):
    mycalc=Calculator(engine,engine)
    b=chess.Board(fen)
    if iswhite is None:
        iswhite= b.turn
    display( widgets.HTML(str(b._repr_svg_())))
    mycalc.print_stats(b,iswhite)
    

In [ ]:
display_board2(fen)